In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import sympy as sp
import at
import math
from tqdm import tqdm

# Selfmade files
from functions import *
from PID import *

# Import ring infoQ_xation
from rings.p3_elements_v24_c4l import *
# ring = load_mat('rings/hmba.mat')

In [ ]:
# Lattice creation
lat = at.Lattice(ring, energy=6.e9)
lat.radiation_off()

lat2 = at.Lattice(ring, energy=6.e9)
lat2.radiation_off() 

# Obtaining Ids
cor_ids = at.get_refpts(lat, at.elements.Corrector)
bpm_ids = at.get_refpts(lat, at.elements.Monitor)
quad_ids = at.get_refpts(lat, at.elements.Quadrupole)

# BPMs positions
s_pos = lat.get_s_pos(bpm_ids)

In [ ]:
dkick = 1e-6
offset = 1e-6
Q_x, Q_y, R_x, R_y = response_matrices(lat, dkick, offset, 1)

In [10]:
import colorednoise as cn

# Time configuration 
# f_turn = 150
f_turn = 130435
fs = f_turn
ts = 1/fs
t_max = 10
t = np.arange(int(fs*t_max))/fs
N = int(fs*t_max)
freqs = np.fft.fftfreq(N, 1/fs)

# A = 4e-6 # um^2/(m*s)
# B = 3e-2 # um^2*Hz^3 proportionally constant charasteristic of the site
A = 1
B = 1
samples = N # number of samples to generate
perturbation_samples = np.zeros((len(quad_ids), samples))

# Quadrupoles positions
quad_pos = lat.get_s_pos(quad_ids)

perturbation_samples[0, :] = cn.powerlaw_psd_gaussian(4, samples) * B

# for i in tqdm(range(1, len(quad_ids))):  
#         L = quad_pos[i] - quad_pos[0]
#         wc = np.sqrt(B/(A*L))
#         b_hp, a_hp = signal.butter(1, wc, 'hp', fs) # High pass filter
#         b_lp, a_lp = signal.butter(1, wc, 'low', fs) # Low pass filter
#         perturbation_samples[i, :] = signal.filtfilt(b_lp, a_lp, perturbation_samples[0, :]) + signal.filtfilt(b_hp, a_hp, cn.powerlaw_psd_gaussian(2, samples) * A * L)

# Correction of magnets position

for i in tqdm(range(0, len(quad_ids))):
        perturbation_samples[i, :] = perturbation_samples[i, :] - ((perturbation_samples[-1, :] - perturbation_samples[0, :]) / (quad_pos[-1] - quad_pos[0])) * (quad_pos[i] - quad_pos[0])

100%|██████████| 417/417 [00:48<00:00,  8.65it/s]


In [1]:
# print(perturbation_samples[:, :])
plt.plot(t, perturbation_samples[0, :])

plt.figure()

plt.loglog(freqs[:N//2], (2*(np.abs(np.fft.fft((1e6 * perturbation_samples[0, :])/N))**2))[:N//2])
plt.xlabel('Frequency [Hz]')
plt.ylabel('PSD [um^2/Hz]')
plt.savefig('power_spectrum.eps', format='eps')


NameError: name 'plt' is not defined

In [ ]:
# Inverse of reponse matrix 
# n_sv = 1000
# R_x_inv, sinv_x = svd_solve(R_x, n_sv, True)
# R_y_inv, sinv_y = svd_solve(R_y, n_sv, True)


# plt.loglog(sinv_x, 'b-', label = 'Horizontal')
# plt.loglog(sinv_y, 'r-', label = 'Vertical')
# plt.legend()
# plt.grid()
# plt.xlabel(r'$s_i$')
# plt.ylabel(r'$|s_i|$')
# plt.savefig('singular_values.eps', format='eps')

In [ ]:
# BPM readings due to an unit actuation
# fig, (ax1, ax2) = plt.subplots(2, 1, constrained_layout=True)
# ax1.plot(s_pos, R_x[1, :] * dkick,'b-', label = "Horizontal")
# ax2.plot(s_pos, R_y[1, :] * dkick,'r-', label = "Vertical")
# ax1.grid()
# ax2.grid()
# ax1.set_title('Horizontal')
# ax2.set_title('Vertical')
# ax1.set_xlabel('BPM position [m]')
# ax1.set_ylabel('Measurement [m]')
# ax2.set_xlabel('BPM position [m]')
# ax2.set_ylabel('Measurement [m]')
# fig.savefig('bpm_unit_actuation.eps', format='eps')

In [ ]:
%matplotlib inline

# TRANSFER FUNCTIONS
# Low pass filter 
wc = 2 * np.pi * 720
num_lp, den_lp = signal.butter(1, wc, 'hp', fs) # High pass filter

# PID controller
Kp = 0.9
Ki = 0.5 * fs
Kd = 0.15 / fs
num_pid, den_pid = PID_transfer_function(Kp, Ki, Kd)

# Simulation configuration
quad_offset = np.zeros((t.size, len(quad_ids)))
corr_str = np.zeros((t.size, len(cor_ids)))
corr_str_delay = np.zeros((t.size, len(cor_ids)))
orbit = np.zeros((t.size, len(bpm_ids)))
orbit_wc = np.zeros((t.size, len(bpm_ids)))
error = np.zeros((t.size, len(bpm_ids)))

reference_orbit = 0
rms = []
rms_wc = []
x_lp = np.zeros(len(cor_ids)*(len(den_lp)-1))
x_pid = np.zeros(len(cor_ids)*(len(den_pid)-1))

# Delay configuration
# loop_delay = 0
# loop_delay = 3e-6
loop_delay = 131e-6
delay_offset = math.ceil(loop_delay*fs) 
print(delay_offset)

# Inverse of reponse matrix 
n_sv = 80
R_x_inv, sinv = svd_solve(R_x, n_sv)

# TEMPORAL TO DELETE
mov = []
rec = []

# Simulation
for n in tqdm(range(1, t.size)):
    # Error calculation
    error[n, :] = reference_orbit - orbit[n-1, :]

    # Correctors str
    corr_str[n, :] = np.dot(R_x_inv.T, error[n, :])

    # PID controller
    corr_str[n, :], x_pid = apply_f(num_pid, den_pid, corr_str[n, :], x_pid, ts)

    # Correctors response
    if n >= delay_offset:
        corr_str_delay[n, :] = corr_str[n - delay_offset, :]

    # Low-pass filter
    filter_out, x_lp = apply_f(num_lp, den_lp, corr_str_delay[n, :] , x_lp, ts)

    # Adding perturbation 
    # quad_offset[n, :] = quad_offset[n, :] + perturbation_samples[n]
    for i in range(0, len(quad_ids)):   
        quad_offset[n, i] = quad_offset[n, i] + perturbation_samples[i, n] 
   
    # Orbit calculation
    orbit_wc[n, :] = np.dot(Q_x.T, quad_offset[n, :]) 
    orbit[n, :] = np.dot(Q_x.T, quad_offset[n, :]) + np.dot(R_x.T,  filter_out)
    
    mov.append(np.dot(Q_x.T, quad_offset[n, :]) [0])
    rec.append(np.dot(R_x.T, corr_str_delay[n, :])[0])
    rms_wc.append(np.sqrt(np.mean(np.square(orbit_wc[n, :]))))
    rms.append(np.sqrt(np.mean(np.square(orbit[n, :]))))

In [ ]:
# Plotting
PSD_WC = np.abs(np.fft.fft(1e6 * orbit_wc[:, 0], norm = 'forward')**2)
PSD = np.abs(np.fft.fft(1e6 * orbit[:, 0], norm = 'forward')**2)

plt.loglog(freqs[:N//2], PSD_WC[:N//2], 'b-', label="FOFB off")
plt.loglog(freqs[:N//2], PSD[:N//2], 'r-', label="FOFB on")
plt.xlabel('Frequency [Hz]')
plt.ylabel('PSD [um^2/Hz]')
plt.legend()
plt.grid()

plt.figure()

IPSD_WC = np.zeros(N//2)
IPSD = np.zeros(N//2)
IPSD_WC[0] = np.sqrt(PSD_WC[0])
IPSD[0] = np.sqrt(PSD[0])
for F in range(1, N//2):
    IPSD_WC[F] = np.sqrt(2*PSD_WC[F]+(IPSD_WC[F-1]**2))
    IPSD[F] = np.sqrt(2*PSD[F]+(IPSD[F-1]**2))
    
plt.loglog(freqs[:N//2], IPSD_WC, 'bx-', label="FOFB off")
plt.loglog(freqs[:N//2], IPSD, 'ro-', label="FOFB on")
plt.xlabel('Frequency [Hz]')
plt.ylabel('Integrated PSD')
plt.xscale('log')
plt.legend()
plt.grid()

plt.figure()

plt.plot(t[:len(t)-1], rms_wc, 'b-', label="FOFB off")
plt.plot(t[:len(t)-1], rms, 'r-', label="FOFB on")
plt.xlabel('Time [s]')
plt.title('RMS')
plt.grid()

plt.figure()

au = []
au_wc = []
for cont in range(0, t.size):
    au_wc.append(np.sum(rms_wc[:cont]))
    au.append(np.sum(rms[:cont]))
plt.plot(t, au_wc, 'b.-', label="FOFB off")
plt.plot(t, au, 'r.-', label="FOFB on")
plt.xlabel('Time [s]')
plt.legend()
plt.title('Cumulative RMS')
plt.grid()



# ****************************************************************

In [ ]:
# Plotear los SVD

plt.figure()

plt.plot(t, perturbation_samples[:] * 1e6)
plt.xlabel('Time [s]')
plt.ylabel('Displacement [um]')
plt.title('Ground motion')
plt.grid()

plt.figure()

plt.loglog(freqs[:N//2], (2*(np.abs(np.fft.fft((1e6 * perturbation_samples)/N))**2))[:N//2])
plt.xlabel('Frequency [Hz]')
plt.ylabel('PSD of ground motion [um^2/Hz]')
plt.grid()

plt.figure()

plt.plot(t[:len(t)-1], mov[:], 'b.', label="Perturbation effect")
plt.plot(t[:len(t)-1], rec[:], 'r.', label="Correctors effect")
res = np.zeros(len(mov))
for i in range(0, len(mov)):
    res[i] = mov[i] + rec[i]
plt.plot(t[:len(t)-1], res[:], 'y.', label="Orbit")
plt.xlabel('Time [s]')
plt.legend()
plt.grid()

plt.figure()

plt.loglog(freqs[:N//2], (2*(np.abs(np.fft.fft((1e6 * orbit_wc[:, 0])/N))**2))[:N//2], 'b-', label="FOFB off")
plt.loglog(freqs[:N//2], (2*(np.abs(np.fft.fft((1e6 * orbit[:, 0])/N))**2))[:N//2], 'r-', label="FOFB on")
plt.xlabel('Frequency [Hz]')
plt.ylabel('PSD [um^2/Hz]')
plt.legend()
plt.grid()

plt.figure()

aux = []
aux2 = []
for f in range(0, N//2):
    aux.append(np.sqrt(np.sum((2*(np.abs(np.fft.fft((1e6 * orbit_wc[:, 0])/N))**2))[:f])))
    aux2.append(np.sqrt(np.sum((2*(np.abs(np.fft.fft((1e6 * orbit[:, 0])/N))**2))[:f])))
plt.loglog(freqs[:N//2], aux, 'bx-', label="FOFB off")
plt.loglog(freqs[:N//2], aux2, 'ro-', label="FOFB on")
plt.xlabel('Frequency [Hz]')
plt.ylabel('Integrated PSD')
plt.xscale('log')
plt.legend()
plt.grid()

plt.figure()

color = iter(plt.cm.rainbow(np.linspace(0, 1, int(t.size/10))))
for n in range(1, t.size):
    if n%10 == 1:
        c = next(color)
        plt.plot(s_pos, 1e6 * orbit[n, :], c=c, label=n)
# plt.legend(loc='best')
plt.ylabel('Position [um]')
plt.xlabel('Time [s]')
plt.title('Closed orbit')
plt.grid()

plt.figure()

plt.plot(t[:len(t)-1], rms_wc, 'b-', label="FOFB off")
plt.plot(t[:len(t)-1], rms, 'r-', label="FOFB on")
plt.xlabel('Time [s]')
plt.title('RMS')
plt.grid()

plt.figure()

au = []
au_wc = []
for cont in range(0, t.size):
    au_wc.append(np.sum(rms_wc[:cont]))
    au.append(np.sum(rms[:cont]))
plt.plot(t, au_wc, 'b.-', label="FOFB off")
plt.plot(t, au, 'r.-', label="FOFB on")
plt.xlabel('Time [s]')
plt.legend()
plt.title('Cumulative RMS')
plt.grid()

In [ ]:
# ts = 1/150
# wc = 2 * np.pi * 715
# num = [wc]
# den = [1, wc]
Kp = 1
Ki = 1
Kd = 2
num_pid, den_pid = PID_transfer_function(Kp, Ki, Kd)

wc = 2 * np.pi * 30
num_lp = [wc]
den_lp = [1, wc]
    
sys = signal.TransferFunction(num_lp, den_lp)

w , mag, phase = signal.bode(sys)
plt.plot(w, mag)
plt.figure()
plt.plot(w, phase)

In [ ]:
%matplotlib inline

# Time configuration 
fs = 150
ts = 1/fs
t_max = 3
t = np.arange(int(fs*t_max))/fs
N = int(fs*t_max)

# Perturbations
# perturbation_samples = spectrum_noise(lambda f: 1 / f**0, N, fs) 

# fmin=1e-3
# fmax=75
# perturbation_samples = sinesweep(t, fmin, fmax)

# fsin = 10
# amplitude = 1
# perturbation_samples = amplitude*np.sin(2*np.pi*t*fsin)

t0 = 0.1
perturbation_samples = np.piecewise(t, [t < t0, t >= t0], [0, 0.0001])

# TRANSFER FUNCTIONS
# Low pass filter 
wc = 2 * np.pi * 200
num_lp = [wc]
den_lp = [1, wc]

# PID controller
Kp = 0
Ki = 1
Kd = 0
num_pid, den_pid = PID_transfer_function(Kp, Ki, Kd)

# Simulation configuration
quad_offset = np.zeros((t.size, len(quad_ids)))
quad_offset2 = np.zeros((t.size, len(quad_ids)))
corr_str = np.zeros((t.size, len(cor_ids)))
corr_str_delay = np.zeros((t.size, len(cor_ids)))
orbit = np.zeros((t.size, len(bpm_ids)))
error = np.zeros((t.size, len(bpm_ids)))

reference_orbit = 0
rms = []
x_lp = np.zeros(len(quad_ids)*(len(den_lp)-1))
x_pid = np.zeros(len(bpm_ids)*(len(den_pid)-1))

# Delay configuration
loop_delay = 131e-3
delay_offset = math.ceil(loop_delay*fs) 

# Inverse of reponse matrix 
n_sv = 80
R_x_inv = svd_solve(R_x, n_sv)

# Simulation
for n in tqdm(range(1, t.size)):
    quad_offset[n, :] = quad_offset[n, :] + perturbation_samples[n]
    quad_offset2[n, :], x_lp = apply_f(num_lp, den_lp, quad_offset[n, :], x_lp, ts)

freqs = np.fft.fftfreq(N, 1/fs)
plt.plot(freqs[:N//2], np.abs(np.fft.fft(perturbation_samples))[:N//2]*(2/len(perturbation_samples)))
plt.xlabel('Frequency [Hz]')
plt.title('Perturbation PSD')

plt.figure()

plt.plot(freqs[:N//2], np.abs(np.fft.fft(quad_offset2[:, 0]))[:N//2]*(2/len(quad_offset2[:, 0])))
plt.xlabel('Frequency [Hz]')
plt.title('Error PSD in element 0')

In [ ]:
def plot_spectrum(s):
    f = np.fft.rfftfreq(len(s))
    plt.loglog(f, np.abs(np.fft.rfft(s)))

def noise_psd(N, psd = lambda f: 1):
        X_white = np.fft.rfft(np.random.randn(N))
        S = psd(np.fft.rfftfreq(N))
        # Normalize S
        S = S / np.sqrt(np.mean(S**2))
        X_shaped = X_white * S
        return np.fft.irfft(X_shaped)

def PSDGenerator(f):
    return lambda N: noise_psd(N, f)

@PSDGenerator
def pink_noise(f):
    return 1/np.where(f == 0, float('inf'), np.sqrt(f))

plt.figure(figsize=(8, 8))
for G in [pink_noise]:
    plot_spectrum(G(2**14))
plt.ylim([1e-3, None])

In [ ]:
quad_offset = np.zeros((t.size, len(quad_ids)))
orbit = np.zeros((t.size, len(bpm_ids)))
corr_str = np.zeros((t.size, len(cor_ids)))

# Inverse of reponse matrix 
# n_sv = 80
# R_x_inv = svd_solve(R_x, n_sv)

# quad_offset[0, :] = quad_offset[0, :] + perturbation_samples[100]
# orbit[0, :] = np.dot(Q_x.T, quad_offset[0, :]) 

# corr_str[0, :] = np.dot(R_x_inv.T, -1 * orbit[0, :])

# quad_offset[1, :] = quad_offset[1, :] + perturbation_samples[100]
# orbit[0, :] = np.dot(Q_x.T, quad_offset[1, :])
# plt.plot(orbit[0, :], 'b.-')

# orbit[0, :] = orbit[0, :] + np.dot(R_x.T, corr_str[0, :]) 
# plt.plot(orbit[0, :], 'r.-')

wc = 2 * np.pi * 10
num_lp = [1]
den_lp = [1, 0]
orbit2 = np.zeros((t.size, len(bpm_ids)))
x_lp = np.zeros(len(bpm_ids)*(len(den_lp)-1))

for a in range(0, t.size):
    orbit[a, :], x_lp = apply_f(num_lp, den_lp, perturbation_samples[a], x_lp, ts)
    # orbit2[a, :] = np.dot(Q_x.T, quad_offset[a, :] + perturbation_samples[a])

# plt.plot(t, orbit2[:, 0], 'b.-')
plt.plot(t, orbit[:, 0], 'r.-')

In [ ]:
# Time configuration 
# f_turn = 150
# # f_turn = 130435
# fs = f_turn
# ts = 1/fs
# t_max = 10
# t = np.arange(int(fs*t_max))/fs
# N = int(fs*t_max)
# freqs = np.fft.fftfreq(N, 1/fs)


# def spectrum_noise(spectrum_func, samples=1024, rate=44100):
#     freqs = np.fft.rfftfreq(samples, 1.0/rate)            # real-fft frequencies (not the negative ones)
#     spectrum = np.zeros_like(freqs, dtype='complex')      # make complex numbers for spectrum
#     spectrum[1:] = spectrum_func(freqs[1:])               # get spectrum amplitude for all frequencies except f=0
#     phases = np.random.uniform(0, 2*np.pi, len(freqs)-1)  # random phases for all frequencies except f=0
#     spectrum[1:] *= np.exp(1j*phases)                     # apply random phases
#     noise = np.fft.irfft(spectrum)                        # return the reverse fourier transform
#     noise = np.pad(noise, (0, samples - len(noise)), 'constant') # add zero for odd number of input samples
 
#     return noise
 
# def pink_spectrum(f, f_min=0, f_max=np.inf, att=np.log10(2.0)*10):
#     s = f**-( 0.5 * (att/10.0) / np.log10(2.0) )  # apply attenuation
#     s[np.logical_or(f < f_min, f > f_max)] = 0    # apply band pass
#     return s

# Perturbations

# fmin=1e-6
# fmax=1
# perturbation_samples = sinesweep(t, fmin, fmax, amplitude = 0.0001)

# ******

# perturbation_samples = real_perturbation(t) * 1e-5

# ******

# fsin = 100
# amplitude = 1
# perturbation_samples = amplitude*np.sin(2*np.pi*t*fsin) + 1

# ******

# t0 = 0.1
# perturbation_samples = np.piecewise(t, [t < t0, t >= t0], [0, 0.00001])

# ******

# import colorednoise as cn

# beta = 2 # the exponent
# samples = N # number of samples to generate
# perturbation_samples = np.zeros((len(quad_ids), samples))
# for i in range(0, len(quad_ids)):   
#         perturbation_samples[i, :] = cn.powerlaw_psd_gaussian(beta, samples) * 10e-6


# ******

# perturbation_samples = spectrum_noise(lambda x:pink_spectrum(x, 0, 1000), N, fs) * 10e-6
# perturbation_samples = spectrum_noise(lambda f: 1 / f**4, N, fs) * 10e-6

# ******

# plt.plot(t, perturbation_samples)

# plt.figure()

# plt.loglog(freqs[:N//2], (2*(np.abs(np.fft.fft((1e6 * perturbation_samples)/N))**2))[:N//2])
# plt.xlabel('frequency [Hz]')
# plt.ylabel('PSD [um^2/Hz]')

# plt.figure()

# aux = []
# for f in range(0, N//2):
#     aux.append(np.sqrt(np.sum((2*(np.abs(np.fft.fft((1e6 * perturbation_samples)/N))**2))[:f])))
# plt.plot(freqs[:N//2], aux, 'x-')
# plt.xlabel('Frequency [Hz]')
# plt.ylabel('Integrated PSD')
# plt.xscale('log')


# plt.plot(freqs, 2 * np.abs(np.fft.fft(perturbation_samples, norm = 'forward')**2))
# plt.xlabel('Frequency [Hz]')
# plt.ylabel('PSD [m/Hz]')

# plt.figure()

# IPSD = np.zeros(N//2)
# PSD = np.abs(np.fft.fft(1e6 * perturbation_samples, norm = 'forward')**2)
# IPSD[0] = np.sqrt(PSD[0])
# for F in range(1, N//2):
#     IPSD[F] = np.sqrt(2*PSD[F]+(IPSD[F-1]**2))
# plt.loglog(freqs[:N//2], PSD[:N//2], 'b-')
# plt.figure()
# plt.plot(freqs[:N//2], IPSD[:N//2], 'r-')
# plt.xlabel('frequency [Hz]')
# plt.ylabel('Integrated PSD')
# plt.xscale('log')